<a href="https://colab.research.google.com/github/Shibika-Roy/Terrain-Recognition/blob/main/hybrid_terrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Here’s the code for the CNN model combined with sensor data fusion using TensorFlow Keras.**

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Concatenate

# Define the CNN model for image input
def create_image_model(input_shape):
    image_input = Input(shape=input_shape)

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(image_input)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)

    return Model(inputs=image_input, outputs=x)

# Define the model for sensor input (e.g., accelerometer, gyroscope)
def create_sensor_model(input_shape):
    sensor_input = Input(shape=input_shape)

    y = Dense(64, activation='relu')(sensor_input)
    y = Dense(32, activation='relu')(y)

    return Model(inputs=sensor_input, outputs=y)

# Combine image and sensor models
def create_hybrid_model(image_shape, sensor_shape):
    image_model = create_image_model(image_shape)
    sensor_model = create_sensor_model(sensor_shape)

    # Concatenate both image and sensor outputs
    combined = Concatenate()([image_model.output, sensor_model.output])

    z = Dense(64, activation='relu')(combined)
    z = Dense(5, activation='softmax')(z)  # Assuming 5 terrain classes

    hybrid_model = Model(inputs=[image_model.input, sensor_model.input], outputs=z)

    return hybrid_model

# Define input shapes (image and sensor data)
image_shape = (64, 64, 3)  # Image: 64x64 size, RGB channels
sensor_shape = (6,)         # Sensor: 6 sensor readings (accelerometer, gyroscope)

# Create the hybrid model
model = create_hybrid_model(image_shape, sensor_shape)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model architecture
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64, 64, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 64, 64, 16)     │            448 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 32, 32, 16)     │              0 │ conv2d[0][0]           │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 32, 32, 32)     │          4,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 16, 16, 32)     │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 8192)           │              0 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 6)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │      1,048,704 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 64)             │            448 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │          8,256 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 32)             │          2,080 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 96)             │              0 │ dense_1[0][0],         │
│                           │                        │                │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 64)             │          6,208 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 5)              │            325 │ dense_4[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,071,109 (4.09 MB)

 Trainable params: 1,071,109 (4.09 MB)

 Non-trainable params: 0 (0.00 B)

Explanation:
Image model: The CNN processes the image data using two convolutional layers, followed by a dense layer.
Sensor model: A fully connected neural network processes the sensor data.
Fusion: The outputs of both models are concatenated and processed through additional fully connected layers.
Final layer: The final output is a classification of 5 terrain types using a softmax activation.



In [4]:
import numpy as np

# Generate dummy image data (32 samples of 64x64 RGB images)
image_data = np.random.rand(32, 64, 64, 3)

# Generate dummy sensor data (32 samples with 6 sensor readings)
sensor_data = np.random.rand(32, 6)

# Generate dummy labels for 5 terrain classes
labels = np.random.randint(0, 5, size=(32,))

# Train the model on the dummy data
model.fit([image_data, sensor_data], labels, epochs=3)


Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.2812 - loss: 1.6431
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 786ms/step - accuracy: 0.2188 - loss: 1.7137
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4375 - loss: 1.5615


Export the Model for Edge Computing

In [ ]:
# Save model as a TensorFlow SavedModel format
model.save('terrain_recognition_model')

# Optionally convert to TensorFlow Lite for edge devices
converter = tf.lite.TFLiteConverter.from_saved_model('terrain_recognition_model')
tflite_model = converter.convert()

# Save the TFLite model
with open('terrain_recognition_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model saved for edge computing deployment.")
